# Installs

In [ ]:
!pip install pyLDAvis

In [ ]:
!pip install plsa

In [ ]:
!pip install bnlp_toolkit

In [ ]:
!pip install bnltk

In [ ]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git

# Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import re
import os

import plsa

from sklearn.feature_extraction.text import CountVectorizer

from bnlp.corpus import stopwords as stopwords_1, punctuations, digits
from bnlp.corpus.util import remove_stopwords
from bnltk.tokenize import Tokenizers
from bengali_stemmer.rafikamal2014 import RafiStemmer

In [ ]:
import sys
import matplotlib.pyplot as plt
%matplotlib notebook

from plsa.pipeline import DEFAULT_PIPELINE

# Connect Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/THESIS/PLSA'

!pip install cache-magic
import cache_magic
!mkdir .cache
!ln -s '/content/drive/My Drive/THESIS/PLSA/.cache' /content/.cache

# Get Dataset

In [ ]:
#articles = pd.read_csv('/content/drive/My Drive/Bangla-Article-and-Summary.csv')
articles = pd.read_csv('/content/drive/MyDrive/THESIS/Final Dataset/all_final_reduced.csv', error_bad_lines=False)
print(articles.isnull().sum())
articles

Full_Text    0
dtype: int64


,Full_Text
0,শনিবার রাতে উপজেলার বামনডাঙ্গা থেকে তাদের আটক...
1,"লেখক, শিক্ষক, অনলাইন অ্যাকটিভিস্ট, ভিন্নমতের ..."
2,রাত ১২টা ০১ মিনিটে নয়া পল্টনে বিএনপির কেন্দ্র...
3,গোলাকার পৃথিবীর আবর্তনের কারণে নিউ জিল‌্যান্ড...
4,শনিবার রাতে গুলশান-২ নম্বর গোলচত্বরে থার্টি ফ...
...,...
321691,স্বাস্থ্যকর জীবনযাপনের জন্য সুষম খাবার জরুরি। ...
321692,শীতে সর্দি-কাশি সমস্যা হয়ে থাকে। অনেক সময় দেখা...
321693,বাংলাদেশ প্রকৌশল বিশ্ববিদ্যালয়ে কর্মকর্তা নিয়...
321694,"অনেকের কোথাও ঘুরতে যাওয়ার সময়, দীর্ঘক্ষণ বাসে-..."


In [ ]:
'''#drop Summary Field
articles.drop('Summary', inplace=True, axis=1)
articles.reset_index(drop=True)
articles'''
articles.dropna(inplace=True)
articles.reset_index(drop=True)
articles.isnull().sum()

Full_Text    0
dtype: int64

In [ ]:
random.seed(42)
bangla_news_list = articles.Full_Text.to_list()
print(bangla_news_list[31])
random.shuffle(bangla_news_list)
print(bangla_news_list[31])

 এ কাজে সিএসসিকে সহযোগিতা করবে বুয়েটের বিআরটিসি (ব্যুরো অফ রিসার্চ টেস্টিং অ্যান্ড কনসালটেন্ট)। রোববার এ বিষয়ে সিএসসির সঙ্গে রেলপথ মন্ত্রণালয়ের একটি চুক্তি হয়। রেলভবনে পদ্মা সেতু রেল সংযোগ প্রকল্প পরিচালক সুকুমার ভৌমিক, সেনাবাহিনী ও বিআরটিসির পক্ষ থেকে কর্নেল মো. আবুল কালাম আজাদ (প্রজেক্ট ম্যানজার, সিএসসি, পদ্মা সেতু রেল সংযোগ প্রকল্প) চুক্তিতে স্বাক্ষর করেন। পদ্মা সেতু রেল সংযোগ প্রকল্পের জন্য সিএসসিকে সুপারভিশন পরামর্শক হিসেবে নিয়োগের বিষয়টি গত ২০ ডিসেম্বর সরকারি ক্রয় সংক্রান্ত মন্ত্রিসভা কমিটির সায় পায়। এ কাজে ব্যয় ধরা হয়েছে ৯৪১ কোটি টাকা। পরামর্শক সেবাকাল দুই হাজার ১০০ দিন। এই সেবার আওতায় সেনাবাহিনীর ইঞ্জিনিয়ারিং কোরের সদস্যরা ঠিকাদারের দাখিল করা নকশা যাচাই করে তার অনুমোদন দেবেন। এছাড়া রেলের জন্য ভূমি অধিগ্রহণ, সাইট ক্লিয়ারেন্স এবং পূর্নবাসন কার্যক্রমে সহায়তা ও তদারকি করবে। দেশি-বিদেশি বিশেশজ্ঞের সমন্বয়ে নির্মাণকাজ সুষ্ঠু তদারকিসহ পরিবেশের ভারসাম্য রক্ষা, আধুনিক প্রযুক্তির ব্যবহার এবং এ সংক্রান্ত প্রয়োজনীয় প্রশিক্ষণসহ যোগ্যতা বৃদ্ধি করা পরামর্শক সেবার মধ্যে রয়েছে। চুক্তিস্বাক্ষর অ

# Pre-process

In [ ]:
stopwords_2 = "অবশ্য অনেক অনেকে অনেকেই অন্তত অথবা অথচ অর্থাত অন্য আজ আছে আপনার আপনি আবার আমরা আমাকে আমাদের আমার আমি আরও আর আগে আগেই আই অতএব আগামী অবধি অনুযায়ী আদ্যভাগে এই একই একে একটি এখন এখনও এখানে এখানেই এটি এটা এটাই এতটাই এবং একবার এবার এদের এঁদের এমন এমনকী এল এর এরা এঁরা এস এত এতে এসে একে এক এ ঐ ই ইহা ইত্যাদি উনি উপর উপরে উচিত ও ওই ওর ওরা ওঁর ওঁরা ওকে ওদের ওঁদের ওখানে কত কবে করতে কয়েক কয়েকটি করবে করলেন করার কারও করা করি করিয়ে করার করাই করলে করলেন করিতে করিয়া করেছিলেন করছে করছেন করেছেন করেছে করেন করবেন করায় করে করেই কাছ কাছে কাজে কারণ কিছু কিছুই কিন্তু কিংবা কি কী কেউ কেউই কাউকে কেন কে কোনও কোনো কোন কখনও ক্ষেত্রে খুব গুলি গিয়ে গিয়েছে গেছে গেল গেলে গোটা চলে ছাড়া ছাড়াও ছিলেন ছিল জন্য জানা ঠিক তিনি তিনঐ তিনিও তখন তবে তবু তাঁদের তাঁাহারা তাঁরা তাঁর তাঁকে তাই তেমন তাকে তাহা তাহাতে তাহার তাদের তারপর তারা তারৈ তার তাহলে তিনি তা তাও তাতে তো তত তুমি তোমার তথা থাকে থাকা থাকায় থেকে থেকেও থাকবে থাকেন থাকবেন থেকেই দিকে দিতে দিয়ে দিয়েছে দিয়েছেন দিলেন দু দুটি দুটো দেয় দেওয়া দেওয়ার দেখা দেখে দেখতে দ্বারা ধরে ধরা নয় নানা না নাকি নাগাদ নিতে নিজে নিজেই নিজের নিজেদের নিয়ে নেওয়া নেওয়ার নেই নাই পক্ষে পর্যন্ত পাওয়া পারেন পারি পারে পরে পরেই পরেও পর পেয়ে প্রতি প্রভৃতি প্রায় ফের ফলে ফিরে ব্যবহার বলতে বললেন বলেছেন বলল বলা বলেন বলে বহু বসে বার বা বিনা বরং বদলে বাদে বার বিশেষ বিভিন্ন বিষয়টি ব্যবহার ব্যাপারে ভাবে ভাবেই মধ্যে মধ্যেই মধ্যেও মধ্যভাগে মাধ্যমে মাত্র মতো মতোই মোটেই যখন যদি যদিও যাবে যায় যাকে যাওয়া যাওয়ার যত যতটা যা যার যারা যাঁর যাঁরা যাদের যান যাচ্ছে যেতে যাতে যেন যেমন যেখানে যিনি যে রেখে রাখা রয়েছে রকম শুধু সঙ্গে সঙ্গেও সমস্ত সব সবার সহ সুতরাং সহিত সেই সেটা সেটি সেটাই সেটাও সম্প্রতি সেখান সেখানে সে স্পষ্ট স্বয়ং হইতে হইবে হৈলে হইয়া হচ্ছে হত হতে হতেই হবে হবেন হয়েছিল হয়েছে হয়েছেন হয়ে হয়নি হয় হয়েই হয়তো হল হলে হলেই হলেও হলো হিসাবে হওয়া হওয়ার হওয়ায় হন হোক জন জনকে জনের জানতে জানায় জানিয়ে জানানো জানিয়েছে জন্য জন্যওজে জে বেশ দেন তুলে ছিলেন চান চায় চেয়ে মোট যথেষ্ট টি অতএব অথচ অথবা অনুযায়ী অনেক অনেকে অনেকেই অন্তত অন্য অবধি অবশ্য অর্থাত আই আগামী আগে আগেই আছে আজ আদ্যভাগে আপনার আপনি আবার আমরা আমাকে আমাদের আমার আমি আর আরও ই ইত্যাদি ইহা উচিত উত্তর উনি উপর উপরে এ এঁদের এঁরা এই একই একটি একবার একে এক্ এখন এখনও এখানে এখানেই এটা এটাই এটি এত এতটাই এতে এদের এব এবং এবার এমন এমনকী এমনি এর এরা এল এস এসে ঐ ও ওঁদের ওঁর ওঁরা ওই ওকে ওখানে ওদের ওর ওরা কখনও কত কবে কমনে কয়েক কয়েকটি করছে করছেন করতে করবে করবেন করলে করলেন করা করাই করায় করার করি করিতে করিয়া করিয়ে থাকবেন থাকা থাকায় থাকে থাকেন থেকে থেকেই থেকেও দিকে দিতে দিন দিয়ে দিয়েছে দিয়েছেন দিলেন দু দুই দুটি দুটো দেওয়া দেওয়ার দেওয়া দেখতে দেখা দেখে দেন দেয় দ্বারা ধরা ধরে ধামার নতুন নয় না নাই নাকি নাগাদ নানা নিজে নিজেই নিজেদের নিজের নিতে নিয়ে নিয়ে নেই নেওয়া নেওয়ার নেওয়া নয় পক্ষে পর পরে পরেই পরেও পর্যন্ত পাওয়া পাচ পারি পারে পারেন পি পেয়ে পেয়্র্ প্রতি প্রথম প্রভৃতি প্রযন্ত প্রাথমিক প্রায় প্রায় ফলে ফিরে ফের বক্তব্য বদলে বন বরং বলতে বলল বললেন বলা বলে বলেছেন বলেন বসে হু বা বাদে বার বি বিনা বিভিন্ন বিশেষ বিষয়টি  বেশ বেশি ব্যবহার ব্যাপারে ভাবে ভাবেই মতো মতোই মধ্যভাগে মধ্যে মধ্যেই মধ্যেও মনে মাত্র মাধ্যমে মোট মোটেই যখন যত যতটা যথেষ্ট যদি যদিও যা যাঁর যাঁরা যাওয়া যাওয়ার যাওয়া যাকে যাচ্ছে যাতে যাদের যান যাবে যায় যার যারা যিনি যে যেখানে যেতে যেন যেমন র রকম রয়েছে রাখা রেখে লক্ষ শুধু শুরু সঙ্গে সঙ্গেও সব সবার সমস্ত সম্প্রতি সহ সহিত সাধারণ সামনে সি সুতরাং সে সেই নয় না হয় হয়ে হয়েছে হয়েছিলেন । , - ; ঃ মিথ্যা শুরু"

In [ ]:
stopwords = []

for word in stopwords_1:
  if word not in stopwords:
    stopwords.append(word)

for word in stopwords_2.split():
  if word not in stopwords:
    stopwords.append(word)

len(stopwords)

445

In [ ]:
def preprocess_BN(text, stop_words=stopwords):
  #text = text.split()
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'\<a href', ' ', text)
  text = re.sub(r'&amp;', '', text) 
  text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
  text = re.sub(r'<br />', ' ', text)
  text = re.sub(r'\'', ' ', text)
  text = re.sub(r'।', ' ', text)
  text = re.sub(r'[‘’]', ' ', text)
  text = re.sub(r'[০১২৩৪৫৬৭৮৯123456789]', ' ', text)
  #text = " ".join(text)
  
  tokenizer = Tokenizers()
  stemmer = RafiStemmer()

  #stop_words = load_stop_word()
  

  doc_token = []

  if isinstance(text, str):
    for token in tokenizer.bn_word_tokenizer(text):
      if token not in stop_words and len(token) >= 3:
        stemmed_token = stemmer.stem_word(token)
        if len(stemmed_token) >= 2 and stemmed_token not in stop_words:
          doc_token.append(stemmed_token)


  #text = remove_stopwords(text, stopwords=stopwords)

  '''if isinstance(text, str):
    text = text.split()
  text = stemming_docs(text)'''
  #print(text)
  return " ".join(doc_token)

In [ ]:
# remove symbols, punctuations, and stopwords

pp_bangla_news_list = []
for bangla_news in bangla_news_list:
  #print("Type: {}".format(type(str(bangla_news))))
  #print("Before PP: {}".format(bangla_news))
  pp_bangla_news_list.append(preprocess_BN(bangla_news))
  #print("After PP: {}\n\n".format(pp_bangla_news_list[-1]))

In [ ]:
articles = pd.read_csv('/content/drive/MyDrive/THESIS/Final Dataset/without Category/plsa_stopwords_removed.csv')

In [ ]:
import regex 

def custom_analyzer(text):
    words = regex.findall(r'\w{2,}', text) # extract words of at least 2 letters
    for w in words:
        yield w

In [ ]:
# Vectorize the dataset FOR BANGLA
vectorizer_BN = CountVectorizer(analyzer=custom_analyzer,
                                max_df=0.8, min_df=0.002) #'''max_df=0.5, min_df=2'''
vectorizer_BN.fit_transform(pp_bangla_news_list[:150000])
#docs_BN = torch.from_numpy(vectorizer_BN.fit_transform(pp_bangla_news_list[:75000]).toarray())
#print("Changes Accepted.")
'''vocab_BN = pd.DataFrame(columns=['word', 'index'])
vocab_BN['word'] = vectorizer_BN.get_feature_names()
vocab_BN['index'] = vocab_BN.index'''

vocab = vectorizer_BN.get_feature_names()

In [ ]:
final_docs = []

for doc in pp_bangla_news_list[:150000]:
    doc_token = []

    for token in doc.split():
        if token in vocab:
            doc_token.append(token)
    final_docs.append(" ".join(doc_token))

In [ ]:
print(len(pp_bangla_news_list[534]))
print(len(final_docs[534]))

2187
1738


# pLSA

In [ ]:
!pip install git+https://github.com/banglakit/bengali-stemmer.git

In [ ]:
#import docx
from bengali_stemmer.rafikamal2014 import RafiStemmer
#from bnltk.stemmer import BanglaStemmer
import string
import pandas as pd
import gensim
import csv
import numpy as np

In [ ]:
def read_doc_as_pandasDF(filename):

    data = pd.read_csv(filename, error_bad_lines=False)
    data_text = data[['Full_Text']]

    data_text['index'] = data_text.index
    documents = data_text

    return (documents)

In [ ]:
def punctuation_remover(text):
    BENGALI_PUNCTUATION = string.punctuation + "—।’‘"
    BENGALI_NUMERALS = "০১২৩৪৫৬৭৮৯"
    return text.translate(str.maketrans(' ', ' ', BENGALI_PUNCTUATION+BENGALI_NUMERALS))

In [ ]:
def load_stop_word(stops_dir = '/content/drive/My Drive/THESIS/bn_stopwords.txt'):

    '''stop_directory = stops_dir

    doc = docx.Document(stop_directory)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)'''

    bengali_stop_words = []
    with open(stops_dir, 'r') as f:
      for word in f.readlines():
        bengali_stop_words.append(word)
        
    bengali_stop_words = frozenset(bengali_stop_words)

    return bengali_stop_words

In [ ]:
def preprocess_documents(text):

    '''preprocessed_list_of_docs = []

    tokenizer = Tokenizers()
    stemmer = RafiStemmer()

    stop_words = load_stop_word()
    preprocessed_docs = []

    doc_token = []

    if isinstance(doc, str):
        for token in tokenizer.bn_word_tokenizer(doc):
            if token not in stop_words and len(token) >= 3:
                stemmed_token = stemmer.stem_word(token)
                if len(stemmed_token) >= 2 and stemmed_token not in stop_words:
                    doc_token.append(stemmed_token)


    return doc_token'''

    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'।', ' ', text)
    text = re.sub(r'[‘’]', ' ', text)
    text = re.sub(r'[০১২৩৪৫৬৭৮৯123456789]', ' ', text)
    text = re.sub(r'[a-zA-Z]', ' ', text)
    #text = " ".join(text)
  
    #print("In PP (before stopwords): {}".format(text))
    '''text = text.split(' ')
    text = [word for word in text if word not in stopwords]'''
  
    tokenizer = Tokenizers()
    stemmer = RafiStemmer()

    stop_words = load_stop_word()

    doc_token = []

    if isinstance(text, str):
        for token in tokenizer.bn_word_tokenizer(text):
            if token not in stop_words and len(token) >= 3:
                stemmed_token = stemmer.stem_word(token)
                if len(stemmed_token) >= 2 and stemmed_token not in stop_words:
                    doc_token.append(stemmed_token)


    #text = remove_stopwords(text, stopwords=stopwords)

    '''if isinstance(text, str):
        text = text.split()
    text = stemming_docs(text)'''
    #print(text)
    return doc_token

In [ ]:
def prepare_bag_of_words(processed_docs, dictionary):
    return [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
def prepare_bow_list(bow_corpus, dictionary):

    header_list = list(range(0, len(dictionary)-1))
    all_list = [header_list]

    for each_list in bow_corpus:
        temp_list = [0]*len(dictionary)
        for each_tuple in each_list:
            temp_list[each_tuple[0]] = each_tuple[1]
        all_list.append(temp_list)

    minimal_all_list = []

    minimal_header_list = []

    for i in range(len(dictionary)):
        minimal_header_list.append(dictionary[i])

    minimal_all_list.append(minimal_header_list)

    for each_mini_list in all_list[1:]:
        minimal_all_list.append(each_mini_list)


    return (minimal_all_list)

In [ ]:
def prepare_model_list_presentation(lda_model, dictionary):

    header_list =  [lda_model.id2word[i] for i in range(0, len(dictionary))]
    list_for_csv = [header_list]

    for i in range(lda_model.num_topics):
        each_topic_list = lda_model.get_topic_terms(i, topn=len(dictionary))
        sorted_each_topic_list = sorted(each_topic_list, key=lambda k: k[0])
        just_value_list = [each_tuple[1] for each_tuple in sorted_each_topic_list]

        list_for_csv.append(just_value_list)

    return (list_for_csv)

In [ ]:
def prepare_plsa_model(bow_corpus, n_topics):
    plsa_model_normal = plsa.algorithms.plsa.PLSA(bow_corpus, n_topics=n_topics, tf_idf=False)
    return plsa_model_normal

In [ ]:
np.random.seed(2021)

NUM_TOPICS = 20
CSV_LOCATION = '/content/drive/MyDrive/THESIS/Final Dataset/all_final_randomized.csv'
    
pd_document = read_doc_as_pandasDF(CSV_LOCATION)

smaller_documents = pd_document[:100000]
processed_docs = smaller_documents['Full_Text'].map(preprocess_documents)

NameError: ignored

In [ ]:
'''np.random.seed(2021)

NUM_TOPICS = 20
CSV_LOCATION = '/content/drive/MyDrive/THESIS/Final Dataset/all_final_randomized.csv'
    
pd_document = read_doc_as_pandasDF(CSV_LOCATION)

smaller_documents = pd_document.Full_Text[:100000]
processed_docs = smaller_documents[['Full_Text']].map(preprocess_documents)'''

dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=.01, no_above=0.7)
#, keep_n=100000

bow_corpus = prepare_bag_of_words(processed_docs, dictionary)
plsa_model_normal = prepare_plsa_model(bow_corpus, n_topics=NUM_TOPICS)

plsa_model_result = plsa_model_normal.fit(eps=1e-05, max_iter=50, warmup=5)

AttributeError: ignored

In [ ]:
#bn_stopwords = []
with open('/content/drive/MyDrive/THESIS/bn_stopwords.txt', 'r') as fp:
    stopwords = [word.split()[0] for word in fp.readlines()]
len(stopwords)

446

In [ ]:
def remove_symbols(text):
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    text = re.sub(r'।', ' ', text)
    text = re.sub(r'[‘’]', ' ', text)
    text = re.sub(r'[০১২৩৪৫৬৭৮৯123456789]', ' ', text)

    return text

In [ ]:
def remove_stopwords(text, stop_words=stopwords):
  #text = text.split()
  tokenizer = Tokenizers()
  stemmer = RafiStemmer()

  #stop_words = load_stop_word()
  

  doc_token = []

  if isinstance(text, str):
    for token in tokenizer.bn_word_tokenizer(text):
      if token not in stop_words and len(token) >= 3:
        stemmed_token = stemmer.stem_word(token)
        if len(stemmed_token) >= 2 and stemmed_token not in stop_words:
          doc_token.append(stemmed_token)


  #text = remove_stopwords(text, stopwords=stopwords)

  '''if isinstance(text, str):
    text = text.split()
  text = stemming_docs(text)'''
  #print(text)
  return " ".join(doc_token)

In [ ]:
np.random.seed(2021)

NUM_TOPICS = 20
CSV_LOCATION = '/content/drive/MyDrive/THESIS/Final Dataset/all_final_reduced.csv'
    
pd_document = read_doc_as_pandasDF(CSV_LOCATION)

smaller_documents = pd_document.Full_Text.to_list()[:100000]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
import sys
import csv

csv.field_size_limit(850000)

850000

In [ ]:
from plsa.preprocessors import *
tokenizer = Tokenizers()
CUSTOM_PIPELINE = (
    remove_numbers,
    remove_tags('<[^>]*>'),
    remove_punctuation(string.punctuation),
    tokenizer.bn_word_tokenizer ,
    RemoveStopwords(stopwords),
    remove_short_words(3)
)

In [ ]:
NUM_TOPICS = 20
CSV_LOCATION = '/content/drive/MyDrive/THESIS/Final Dataset/without Category/plsa_final_dataset.csv'
#pipes = (remove_symbols, remove_stopwords)
pipeline = plsa.pipeline.Pipeline(*CUSTOM_PIPELINE)
bow_corpus = plsa.corpus.Corpus.from_csv(CSV_LOCATION, pipeline, encoding='utf-8', max_docs=150000)

In [ ]:
bow_corpus

In [ ]:
plsa_model_normal = prepare_plsa_model(bow_corpus, n_topics=NUM_TOPICS)

plsa_model_result = plsa_model_normal.fit(eps=1e-05, max_iter=50, warmup=5)

In [ ]:
topics_prod = []
for i in range(20):
    ls = [tup[0] for tup in plsa_model_result.word_given_topic[i][:10]]
    topics_prod.append(ls)

In [ ]:
topics_prod

[['পুলিশ',
  'গ্রাম',
  'ঘটনা',
  'ছেল',
  'জানান',
  'উপজেল',
  'হাসপাতাল',
  'থান',
  'উদ্ধ',
  'লাশ'],
 ['রান',
  'ম্যাচ',
  'বাংলাদেশ',
  'উইকেট',
  'গোল',
  'শেষ',
  'তিন',
  'মিনিট',
  'ইনিংস',
  'ওভার'],
 ['শেখ',
  'প্রধানমন্ত্রী',
  'বাংলাদেশ',
  'দেশ',
  'হাসিনা',
  'কথা',
  'সাল',
  'মানুষ',
  'বঙ্গবন্ধু',
  'জাতী'],
 ['মামলা',
  'আদালত',
  'সাল',
  'আবেদন',
  'শ্রমিক',
  'বছর',
  'আদাল',
  'কোর্ট',
  'আইন',
  'মামল'],
 ['করোনাভাইরাস',
  'আক্রান্ত',
  'মৃত্যু',
  'দেশ',
  'করোনা',
  'রোগী',
  'শনাক্ত',
  'হাসপাতাল',
  'সংখ্যা',
  'লাখ'],
 ['সম্পাদক',
  'সভাপতি',
  'কমি',
  'সদস্য',
  'চৌধুরী',
  'রহমান',
  'নেতা',
  'হোস',
  'জাতী',
  'জেলা'],
 ['গান',
  'শুভ',
  'ভালো',
  'জীবন',
  'চলচ্চিত্র',
  'অভিন',
  'নাটক',
  'নাম',
  'আপনা',
  'করুন'],
 ['যুক্তরাষ্ট্র',
  'প্রেসিডেন্ট',
  'ট্রাম্প',
  'মার্কিন',
  'সাল',
  'তুরস্ক',
  'দেশ',
  'কিম',
  'অ্যাপ',
  'বছর'],
 ['লীগ',
  'আওয়ামী',
  'নির্বাচন',
  'প্রার্থী',
  'ভোট',
  'বিএনপি',
  'নেতা',
  'বিএনপির',
  'সম্পাদক',
  'ওয়ার্

In [ ]:
final_articles_list = pd.read_csv('/content/drive/MyDrive/THESIS/Final Dataset/without Category/plsa_final_dataset.csv').Full_Text.to_list()

In [ ]:
i = 0
for doc in final_articles_list:
    if i == 5:
        break
    i += 1
    print(doc.split())

['তৃণমূল', 'যোগ', 'দক্ষিণ', 'বিধানসভা', 'কেন্দ্র', 'প্রার্থী', 'সোশ্যাল', 'মিডিয়', 'শিক', 'অভিনেত্রী', 'ঘোষ', 'সোশ্যাল', 'মিডিয়া', 'বিজেপি', 'আক্রমণ', 'ছাড়', 'ঘোষ', 'আগ', 'কথা', 'বিজেপির', 'মুখ', 'খুল', 'আক্রমণ', 'মুখ', 'পড়', 'ঘোষ', 'ঘটন', 'বিজেপি', 'নেতা', 'ঘোষ', 'টুইট', 'যুদ্ধ', 'শনিব', 'বিজেপি', 'খোলা', 'চিঠি', 'লিখ', 'চিঠির', 'কষ্ট', 'শান্তি', 'থাক', 'লম্বা', 'পোস্ট', 'লিখ', 'বিজেপি', 'আপনা', 'অ্যাটাক', 'ক্যাম্পেইন', 'চলচ্চিত্র', 'জগত', 'মানুষ', 'মানুষ', 'ব্যক্তি', 'সম্পর্', 'ভুল', 'তৈরি', 'আপনা', 'রাখ', 'বাংল', 'পুরনো', 'বাংল', 'মানুষ', 'ভালোবাসা', 'প্রতিষ্ঠিত', 'বোঝা', 'আপনা', 'একটু', 'অস্বস্তি', 'পড়', 'মানুষ', 'পরিষ্ক', 'দি', 'মানুষ', 'মেয়ে', 'সম্মান', 'আপনা', 'ধা', 'থাকব', 'আপনা', 'দল', 'নেতা', 'দেবী', 'প্রশ্ন', 'দল', 'নারী', 'সম্মান', 'কথা', 'আপনা', 'অস্বস্তি', 'পড়ব', 'এছাড়া', 'বাংল', 'মানুষ', 'মনোভাব', 'মুখ', 'ভাষা', 'আপনা', 'মেলা', 'আপনা', 'দল', 'নেতা', 'পরিবর্তন', 'স্লোগান', 'শেষ', 'লিখ', 'ঘোষ', 'দক্ষিণ', 'কেন্দ্র', 'তৃণমূল', 'প্রার্থী', 'ঘোষণা', 'মমতা', 'বন্দ্যোপাধ্যা

In [ ]:
new_doc = final_articles_list[52834:52835]

topic_components, number_of_new_words, new_words = plsa_model_result.predict(new_doc[0])

print('Relative topic importance in new document:\n', topic_components)
print('Number of previously unseen words in new document:', number_of_new_words)
print('Previously unseen words in new document:', new_words)

Relative topic importance in new document:
 [0.03228734 0.02794212 0.09324692 0.08562618 0.0295061  0.13685694
 0.03450409 0.04901984 0.14699766 0.0283752  0.02069639 0.03340841
 0.02356898 0.02509774 0.02469394 0.02015825 0.12767861 0.01704737
 0.03311987 0.01016806]
Number of previously unseen words in new document: 0
Previously unseen words in new document: ()


In [ ]:
tokenized_corpus = []
remove_index = []

for i, doc in enumerate(final_articles_list):
    if isinstance(doc, str):
        tokenized_corpus.append(doc.split())
    else:
        remove_index.append(doc)

In [ ]:
len(remove_index)

0

In [ ]:
for doc in final_articles_list:
    if not isinstance(doc, str):
        print(doc)

nan
nan


In [ ]:
for doc in remove_index:
    final_articles_list.remove(doc)

In [ ]:
print(len(tokenized_corpus))
print(len(final_articles_list))

149998
149998


In [ ]:
from gensim.corpora import Dictionary
#corpus_ = docs_BN.detach().cpu().numpy() # .cpu().numpy().arra
#corpus_ model=lda_model
dct = Dictionary(tokenized_corpus)

In [ ]:
from gensim.models import CoherenceModel
# from gensim.corpora import Dictionary
# Compute Coherence Score
coherence_model_lda = CoherenceModel(topics=topics_prod, texts=tokenized_corpus, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.42991018176544626
